In [3]:
using JuMP, HiGHS, DataFrames

In [83]:
generator_DF = DataFrame(
    name = ["A","B","C","D","E"],
    cost = [30, 20, 40, 0, 46],
    capacity = [20,20,60,30,5],
    rate = [1, 0.5, 1.3, 0, 0.45],
    zone = [1,1,2,2,1]
)
colnames = names(generator_DF)#filter(x -> x != "name", names(generator_DF))
gen_data = Dict()
for colname in colnames
    gen_data[colname] = JuMP.Containers.DenseAxisArray(generator_DF[!,colname], generator_DF[!,"name"])
end

g_name = generator_DF.name
demand = [51,49]

2-element Vector{Int64}:
 51
 49

In [206]:
model = Model(HiGHS.Optimizer)
@variables(model, begin
    gen[g_name] >= 0
    flow[g_name] >= 0
    virtual_gen[g_name, zone=1:2] >= 0
end)

@constraints(model, begin
    gen_cap[g in g_name], gen[g] <= gen_data["capacity"][g]
    gen_flow[g in g_name], flow[g] <= gen[g]
    export_priority[g in g_name], flow[g] <= flow["D"]
    demand_balance[zone in [1,2]], sum(virtual_gen[g,zone] for g in g_name)  == demand[zone]
    #demand_balance2, sum(gen[g] for g in g_name) == sum(demand[zone] for zone in [1,2])
    # NO BCA; ADD GEN COST
end)

for generator in g_name
    for zone in [1,2]
        if gen_data["zone"][generator] == zone
            @constraint(model, virtual_gen[generator, zone] == gen[generator]-flow[generator])
        else
            @constraint(model, virtual_gen[generator, zone] == flow[generator])
        end
    end
end

@objective(model, Min, sum(gen_data["cost"][g]*gen[g] for g in g_name) + sum(gen_data["rate"][g]*virtual_gen[g,1]*0 for g in g_name if gen_data["zone"][g] == 1))

optimize!(model)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
16 rows, 15 cols, 43 nonzeros
14 rows, 14 cols, 43 nonzeros
Presolve : Reductions: rows 14(-13); columns 14(-6); elements 43(-15)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 2(100) 0s
         10     2.2000000000e+03 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 10
Objective value     :  2.2000000000e+03
HiGHS run time      :          0.00


In [207]:
value.(virtual_gen)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, ["A", "B", "C", "D", "E"]
    Dimension 2, Base.OneTo(2)
And data, a 5×2 Matrix{Float64}:
 20.0  -0.0
  1.0  19.0
 -0.0  30.0
 30.0   0.0
 -0.0  -0.0